In [ ]:
#default_exp peptide.mass_calc

## Mass calculation functions for fragments, it should be only called by peptide.fragment.py.

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np
from typing import List, Tuple, Iterable

from alphabase.constants.aa import (
    calc_sequence_mass, 
    calc_AA_masses_for_same_len_seqs,
    calc_sequence_masses_for_same_len_seqs
)
from alphabase.constants.modification import (
    calc_modification_mass,
    calc_modification_mass_sum
)
from alphabase.constants.element import MASS_H2O

def calc_delta_modification_mass(
    pep_len:int,
    mass_deltas:List[float],
    mass_delta_sites:List[int]
)->np.array:
    '''
    For open-search, we may also get modification 
    mass deltas other than mod names. This function calculate
    modification masses from these delta masses.
    Args:
        pep_len (int): nAA
        mass_deltas (list of floats): mass deltas on the peptide
        mass_delta_sites (list of int): localized sites of corresponding mass deltas
    Returns:
        np.array: 1-D array with length=`peplen`.
            Masses of modifications (mass deltas) through the peptide,
            `0` if sites has no modifications
    '''
    masses = np.zeros(pep_len)
    for site, mass in zip(mass_delta_sites, mass_deltas):
        if site == 0:
            masses[site] += mass
        elif site == -1:
            masses[site] += mass
        else:
            masses[site-1] += mass
    return masses

def calc_b_y_and_peptide_mass(
    sequence: str,
    mod_names: List[str],
    mod_sites: List[int],
    mass_deltas: List[float] = None,
    mass_delta_sites: List[int] = None,
)->Tuple[np.array,np.array,float]:
    '''
    It is highly recommend to use 
    `calc_b_y_and_peptide_mass_for_same_len_seqs()`
    as it is much faster
    '''
    residue_masses = calc_sequence_mass(sequence)
    mod_masses = calc_modification_mass(
        len(sequence), mod_names, mod_sites
    )
    residue_masses += mod_masses
    if mass_deltas is not None:
        mod_masses = calc_delta_modification_mass(
            len(sequence), mass_deltas, mass_delta_sites
        )
        residue_masses += mod_masses
    #residue_masses = residue_masses[np.newaxis, ...]
    b_masses = np.cumsum(residue_masses)
    b_masses, pepmass = b_masses[:-1], b_masses[-1]
        
    pepmass += MASS_H2O
    y_masses = pepmass - b_masses
    return b_masses, y_masses, pepmass

def calc_peptide_mass_for_same_len_seqs(
    sequences: np.array,
    mod_list: List[str],
    mass_delta_list: List[str]=None
)->np.array:
    '''
    Calculate peptide masses for peptide sequences with same lengths.
    We need 'same_len' here because numpy can process AA sequences 
    with same length very fast. 
    See `alphabase.aa.calc_sequence_masses_for_same_len_seqs()`

    Args:
        mod_list (List[str]): list of modifications, 
            e.g. `['Oxidation@M;Phospho@S','Phospho@S;Deamidated@N']`
        mass_delta_list (List[str]): List of modifications as mass deltas,
            e.g. `['15.9xx;79.9xxx','79.9xx;0.98xx']`
    Returns:
        np.array: peptide masses (1-D array, H2O already added)
    '''
    seq_masses = calc_sequence_masses_for_same_len_seqs(
        sequences
    )
    mod_masses = np.zeros_like(seq_masses)
    for i, mods in enumerate(mod_list):
        if len(mods) != 0:
            mod_masses[i] = calc_modification_mass_sum(
                mods.split(';')
            )
    if mass_delta_list is not None:
        for i, mass_deltas in enumerate(mass_delta_list):
            if len(mass_deltas) != 0:
                mod_masses[i] += np.sum([
                    float(mass) for mass in mass_deltas.split(';')
                ])
    return seq_masses+mod_masses
    

def calc_b_y_and_peptide_mass_for_same_len_seqs(
    sequences: np.array,
    mod_list: Iterable[List[str]],
    site_list: Iterable[List[int]],
    mass_delta_list: Iterable[List[float]]=None,
    mass_delta_site_list: Iterable[List[int]]=None,
)->Tuple[np.array,np.array,np.array]:
    '''
    Calculate b/y fragment masses and peptide masses 
    for peptide sequences with same lengths.
    We need 'same_len' here because numpy can process AA sequences 
    with same length very fast.

    Args:
        sequence (np.array of str): np.array of peptie sequences with same length.
        mod_list (Iterable[List[str]]): Iterable (list, array, ...) of modifications , 
            e.g. `[['Oxidation@M','Phospho@S'],['Phospho@S','Deamidated@N']]` 
        site_list (Iterable[List[int]]): Iterable of modification sites
            corresponding to `mod_list`, e.g. `[[3,6],[4,17]]`
        mass_delta_list (Iterable[List[float]]): Iterable of modifications, 
            e.g. `[[15.994915,79.966331],[79.966331,0.984016]]` 
        mass_delta_site_list (Iterable[List[int]]): Iterable of modification mass delta sites
            corresponding to `mod_list`, e.g. `[[3,6],[4,17]]`
    Returns:
        np.array: neutral b fragment masses (2-D array)
        np.array: neutral y fragmnet masses (2-D array)
        np.array: neutral peptide masses (1-D array)
    '''
    residue_masses = calc_AA_masses_for_same_len_seqs(sequences)
    mod_masses = np.zeros_like(residue_masses)
    seq_len = len(sequences[0])
    for i, (mods, sites) in enumerate(zip(mod_list, site_list)):
        if len(mods) != 0:
            mod_masses[i,:] = calc_modification_mass(
                seq_len, 
                mods, 
                sites,
            )
    if mass_delta_list is not None:
        for i, (mass_deltas, sites) in enumerate(zip(
            mass_delta_list, mass_delta_site_list
        )):
            if len(mass_deltas) != 0:
                mod_masses[i,:] += calc_delta_modification_mass(
                    seq_len, 
                    mass_deltas, 
                    sites,
                )
    residue_masses += mod_masses

    b_masses = np.cumsum(residue_masses, axis=1)
    b_masses, pepmass = b_masses[:,:-1], b_masses[:,-1:]
        
    pepmass += MASS_H2O
    y_masses = pepmass - b_masses
    return b_masses, y_masses, pepmass.reshape(-1)

In [ ]:
seq, mods, mod_sites = 'AGHCEWQMK', ['Carbamidomethyl@C', 'Oxidation@M'], [4, 8]
calc_b_y_and_peptide_mass(seq, mods, mod_sites)

(array([  71.03711379,  128.05857751,  265.11748936,  425.14813804,
         554.19073113,  740.27004408,  868.32862159, 1015.3640213 ]),
 array([1090.43243521, 1033.41097149,  896.35205963,  736.32141095,
         607.27881786,  421.19950491,  293.14092741,  146.1055277 ]),
 1161.46954899713)

In [ ]:
calc_b_y_and_peptide_mass_for_same_len_seqs([seq]*2, [mods,[]], [mod_sites,[]])

(array([[  71.03711379,  128.05857751,  265.11748936,  425.14813804,
          554.19073113,  740.27004408,  868.32862159, 1015.3640213 ],
        [  71.03711379,  128.05857751,  265.11748936,  368.12667432,
          497.16926741,  683.24858036,  811.30715787,  942.34764296]]),
 array([[1090.43243521, 1033.41097149,  896.35205963,  736.32141095,
          607.27881786,  421.19950491,  293.14092741,  146.1055277 ],
        [1017.41605687,  960.39459315,  823.33568129,  720.32649633,
          591.28390324,  405.20459029,  277.14601279,  146.1055277 ]]),
 array([1161.469549  , 1088.45317066]))

In [ ]:
calc_peptide_mass_for_same_len_seqs([seq]*2, [';'.join(mods),""])

array([1161.469549  , 1088.45317066])